<a href="https://colab.research.google.com/github/440g/painkiller/blob/data_preprocessing/Data_preprocessing1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving news_popular_train.csv to news_popular_train.csv


In [32]:
import pandas as pd

# Load data
df = pd.read_csv('news_popular_train.csv')

In [33]:
#Handling missing values
df = df.drop(columns=['shares'])

#전체 numerical 변수 중앙값으로 일괄 처리 후 일부는 평균값 처리
num_cols = df.select_dtypes(include=['float64', 'int64']).columns.drop(['id', 'y'])
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

mean_cols = [
    'global_sentiment_polarity', 'global_rate_positive_words',
    'rate_negative_words', 'avg_negative_polarity', 'min_negative_polarity',
    'title_sentiment_polarity', 'title_subjectivity'
]
for col in mean_cols:
    df[col] = df[col].mask(df[col].isna(), df[col].mean())


#categorical 변수 unknown으로 처리
df['data_channel']=df['data_channel'].fillna('unknown')
df['weekday']=df['weekday'].fillna('unknown')

In [34]:
#categorical 변수 One-hot encoding
df = pd.get_dummies(df, columns=['data_channel', 'weekday'], drop_first=True)

In [35]:
#Handling outliers
import numpy as np

#1. IQR 기반 clip
clip_cols = [
    'n_tokens_content', 'num_hrefs', 'num_self_hrefs',
    'kw_avg_avg']
for col in clip_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df[col] = df[col].clip(lower, upper)

#2. 상한 제한
    df['num_videos'] = df['num_videos'].clip(upper=6)
    df['num_imgs'] = df['num_imgs'].clip(upper=20)

#3. 값 중 51.8%가 -1 => 반 이상 결측값(dummy처리) => 제거
df = df.drop(columns=['kw_min_min'])

#4
df['average_token_length'] = df['average_token_length'].replace(0, np.nan)
df['average_token_length'] = df['average_token_length'].fillna(df['average_token_length'].median())

#5. 음수-> NaN-> 중앙값
df['kw_avg_min'] = df['kw_avg_min'].replace(-1, np.nan)
df['kw_avg_min'] = df['kw_avg_min'].fillna(df['kw_avg_min'].median())
df['kw_avg_min'] = np.log1p(df['kw_avg_min'].clip(lower=0))

#6. kw_min_max, Zeros = 8611 (38.8%) => 결측치의 대체값으로 판단
# 0을 결측치로 보고 중앙값으로 채움
df['kw_min_max'] = df['kw_min_max'].replace(0, np.nan)
df['kw_min_max'] = df['kw_min_max'].fillna(df['kw_min_max'].median())

#7. kw_min_avg, 음수값, Zeros	8664(39.0%) =>음수 처리 및 결측치 대체값으로 판단
df['kw_min_avg'] = df['kw_min_avg'].replace(-1, np.nan)
df['kw_min_avg'] = df['kw_min_avg'].replace(0, np.nan)
df['kw_min_avg'] = df['kw_min_avg'].fillna(df['kw_min_avg'].median())

#8. n_tokens_content==0 594(2.7%)
df = df[df['n_tokens_content'] != 0]

#9.
log_transform_cols = ['num_hrefs', 'kw_max_min', 'kw_min_max', 'kw_max_avg','self_reference_min_shares', 'self_reference_max_shares', 'self_reference_avg_sharess']

for col in log_transform_cols:
  df[col] = np.log1p(df[col])

In [36]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# numerical inputs
num_cols = df.select_dtypes(include=['float64', 'int64']).columns.drop(['id', 'y'])

# scaling
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

X = df.drop(columns=['id', 'y'])
y = df['y']

In [37]:
#Data split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)